<a href="https://colab.research.google.com/github/olgasherbiena/KPI/blob/main/lab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Завдання щодо машинного перекладу на базі рекурентних мереж.
Вирішіть завдання щодо генерації текстів або машинного перекладу. Особливо вітаються україномовні моделі.  

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import os
import requests

In [26]:
input_texts = []  #українська
target_texts = [] #англійська

with open("ukr.txt", "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

# Обробляємо кожен рядок
for line in lines[:10000]:
    parts = line.split("\t")
    if len(parts) >= 2:
        eng_sent = parts[0]
        ukr_sent = parts[1]

        # НАПРЯМОК: ENG -> UKR
        input_texts.append(eng_sent)          # Вхід: Англійська
        target_texts.append("\t" + ukr_sent + "\n")

# 4. Перевірка результату
print(f"Завантажено {len(input_texts)} пар речень.")

Завантажено 10000 пар речень.


In [28]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# --- НАЛАШТУВАННЯ ---
BATCH_SIZE = 64
EPOCHS = 50
LATENT_DIM = 256

# 1. Токенізація ВХОДУ (Англійська)
print("Обробка Англійської...")
input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_texts)
input_sequences = input_tokenizer.texts_to_sequences(input_texts)
input_word_index = input_tokenizer.word_index
max_encoder_seq_length = max([len(x) for x in input_sequences])
num_encoder_tokens = len(input_word_index) + 1

# 2. Токенізація ВИХОДУ (Українська)
print("Обробка Української...")
target_tokenizer = Tokenizer(filters='') # Лишаємо спецсимволи
target_tokenizer.fit_on_texts(target_texts)
target_sequences = target_tokenizer.texts_to_sequences(target_texts)
target_word_index = target_tokenizer.word_index
max_decoder_seq_length = max([len(x) for x in target_sequences])
num_decoder_tokens = len(target_word_index) + 1

# 3. Підготовка матриць
print("Створення матриць...")
encoder_input_data = pad_sequences(input_sequences, maxlen=max_encoder_seq_length, padding='post')
decoder_input_data = pad_sequences(target_sequences, maxlen=max_decoder_seq_length, padding='post')

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, target_seq in enumerate(target_sequences):
    for t, char_index in enumerate(target_seq):
        if t > 0:
            decoder_target_data[i, t - 1, char_index] = 1.0

# 4. Створення моделі
print("Архітектура LSTM...")
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(num_encoder_tokens, LATENT_DIM, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(LATENT_DIM, return_state=True)
_, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, LATENT_DIM, mask_zero=True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(LATENT_DIM, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# 5. Старт навчання
print("Починаємо навчання (50 епох)...")
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.2)

Обробка Англійської...
Обробка Української...
Створення матриць...
Архітектура LSTM...
Починаємо навчання (50 епох)...
Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step - accuracy: 0.0043 - loss: 5.2443 - val_accuracy: 0.0069 - val_loss: 5.0533
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.0047 - loss: 4.7139 - val_accuracy: 0.0069 - val_loss: 4.9223
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.0053 - loss: 4.6103 - val_accuracy: 0.0069 - val_loss: 4.9197
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.0051 - loss: 4.5629 - val_accuracy: 0.0073 - val_loss: 4.9101
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.0059 - loss: 4.5428 - val_accuracy: 0.0064 - val_loss: 4.9080
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.0057 - loss: 4.5337 - val_accuracy: 0.0065 - val_loss: 4.8955
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.0062 - loss: 4.5255 - val_accuracy: 0.00

In [33]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import requests, zipfile, os

def decode_sequence(input_text):
    seq = input_tokenizer.texts_to_sequences([input_text])
    seq = pad_sequences(seq, maxlen=max_enc_len, padding='post')
    states_value = enc_model.predict(seq, verbose=0)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_word_index['\t']
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = dec_model.predict([target_seq] + states_value, verbose=0)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        if sampled_token_index == 0: break
        sampled_word = reverse_target_word_index[sampled_token_index]

        if sampled_word == '\n' or len(decoded_sentence) > 100:
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_word
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]
    return decoded_sentence.strip()

print("\n РЕЗУЛЬТАТИ (ENG -> UKR):")
print("-" * 50)
for eng, _ in demo_pairs:
    # Чистка вхідного тексту
    clean_line = eng.lower().replace(".", "").replace("?", "")
    trans = decode_sequence(clean_line)
    print(f"EN: {eng:<25} | UA: {trans}")

Навчання (30 епох)...
Навчання завершено!

 РЕЗУЛЬТАТИ (ENG -> UKR):
--------------------------------------------------
EN: I go home                 | UA: я йду додому
EN: He is my brother          | UA: він мій брат
EN: She loves him             | UA: вона любить його
EN: Where is the money        | UA: де гроші
EN: I do not want to sleep    | UA: я не хочу спати
EN: It is very cold today     | UA: сьогодні дуже холодно
EN: Open the door             | UA: відкрий двері
EN: Who are you               | UA: хто ти


In [40]:
# --- ТЕСТУВАННЯ (Просто перевіряємо, що модель думає зараз) ---

simple_sentences = [
    "My name is Tom.",
    "I live in London.",
    "London is a big city.",
    "I have a dog and a cat.",
    "My family is very happy.",
    "I go to school every day.",
    "I am happy.",           # Я щасливий
    "He is my brother.",     # Він мій брат
    "She has a cat.",        # У неї є кіт (або "Вона має кота")
    "This is my house.",     # Це мій дім
    "We go home.",           # Ми йдемо додому
    "It is very cold.",      # Дуже холодно
    "I love you.",           # Я тебе люблю
    "the door."         # Відкрий двері
]

print(f"{'INPUT (EN)':<30} | {'OUTPUT (UA)':<30}")
print("-" * 65)

for line in simple_sentences:
    # Очистка тексту (на всяк випадок)
    clean_line = line.lower().replace(".", "").replace("?", "")
    translation = decode_sequence(clean_line)
    print(f"{line:<30} | {translation:<30}")
print("-" * 65)

INPUT (EN)                     | OUTPUT (UA)                   
-----------------------------------------------------------------
My name is Tom.                | зроби це зараз.               
I live in London.              | я почуваюся тома.             
London is a big city.          | в проблем.                    
I have a dog and a cat.        | я це зробив.                  
My family is very happy.       | а зараз і і гаразд.           
I go to school every day.      | я це зробив.                  
I am happy.                    | я не знаю.                    
He is my brother.              | він мій брат                  
She has a cat.                 | вона дуже холодно             
This is my house.              | це це порядку.                
We go home.                    | ми можемо спробувати.         
It is very cold.               | сьогодні дуже холодно         
I love you.                    | я тебе ненавиджу.             
the door.                      | відкр

# **Завдання щодо генерації зображень**

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
import torch

# Завантаження датасету CIFAR-100
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
train_dataset = datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform
)
cifar100_classes = train_dataset.classes
target_class = 60
class_name = cifar100_classes[target_class]
indices = [i for i, (_, y) in enumerate(train_dataset) if y == target_class]
sampler = torch.utils.data.SubsetRandomSampler(indices)
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
print(f"Кількість зображень для навчання: {len(indices)}")